# Load dataset

In [6]:
from datasets import load_from_disk
train = load_from_disk("train.hf")
test = load_from_disk("test.hf")

In [7]:
train = train.shuffle(seed=42)

Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\train.hf\cache-aab69560c24eae2b.arrow


In [8]:
test = test.shuffle(seed=42)

Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\test.hf\cache-6c7c52e5435094d8.arrow


# Processing Bert without cleaning

In [9]:
from transformers import AutoTokenizer
bert_model_name = "HooshvareLab/bert-fa-zwnj-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

In [10]:
text = " ".join(train["words"][0])
bert_tokens = bert_tokenizer(text).tokens()
bert_tokens

['[CLS]',
 'جمعیت',
 'جمعیت',
 'این',
 'دهستان',
 'بر',
 'اساس',
 'سرشماری',
 'سال',
 '۱۳۸۵',
 '(',
 '۱',
 '٬',
 '۲۱۲',
 '##خانوار',
 ')',
 '۵',
 '٬',
 '۱۱۸',
 '##نفر',
 'بوده',
 'است',
 '.',
 '[SEP]']

In [11]:
" ".join(train["words"][0])

'جمعیت جمعیت این دهستان بر اساس سرشماری سال ۱۳۸۵ ( ۱٬۲۱۲خانوار ) ۵٬۱۱۸نفر بوده است .'

## Create a custom model for token classification

In [14]:
from transformers import AutoConfig, AutoModel

bert_model = AutoModel.from_pretrained("HooshvareLab/bert-fa-zwnj-base",cache_dir=Path.cwd())
bert_model = AutoConfig.from_pretrained("HooshvareLab/bert-fa-zwnj-base",cache_dir=Path.cwd())

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
